In [1]:
from pathlib import Path
import sys

sys.path.append(str("/Users/prakharjain/code/aglio_ai/backend"))

from dotenv import load_dotenv

load_dotenv("/Users/prakharjain/code/aglio_ai/backend/.env")

from main import app


/Users/prakharjain/miniconda3/envs/aglio_ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from models.schema import get_db, Restaurant

db = next(get_db())

restaurants = db.query(Restaurant).all()
for restaurant in restaurants:
    print(vars(restaurant))

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x163249fd0>, 'id': 1, 'slug': 'test_nopos', 'name': 'Test NoPOS', 'require_pass': False, 'is_open': True, 'public_id': 'test_nopos', 'tz': 'Asia/Kolkata', 'api_key': 'def0435915cd'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x16324a0f0>, 'id': 2, 'slug': 'petpooja', 'name': 'Cafe PetPooja DineIn', 'require_pass': False, 'is_open': True, 'public_id': 'petpooja_dinein', 'tz': 'Asia/Kolkata', 'api_key': '6ec44b86d479'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x16324a210>, 'id': 3, 'slug': 'petpooja_delivery', 'name': 'Cafe PetPooja Delivery', 'require_pass': False, 'is_open': True, 'public_id': 'petpooja_delivery', 'tz': 'Asia/Kolkata', 'api_key': '8da380c3f6be'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x16324a330>, 'id': 4, 'slug': 'handcrafted', 'name': 'Cafe Handcrafted', 'require_pass': False, 'is_open': True, 'public_id': 'handcraf

In [11]:
Restaurant

models.schema.Restaurant